### 1. 环境设置与导入

安装依赖，并配置系统路径以正确导入所有用于预测、融合和风控的模块。

In [1]:
import pandas as pd
import yaml
import joblib
import torch
import numpy as np
from pathlib import Path
import baostock as bs
from tqdm.notebook import tqdm
import sys
import os

# --- 核心修正：将项目根目录添加到系统路径 ---
project_root = os.getcwd()
if project_root not in sys.path:
    sys.path.append(project_root)
print(f"Project root '{project_root}' added to sys.path.")
# ------------------------------------------

from data_process import get_data
from data_process import data_contracts
from data_process import risk_manager
from model import fusion_strategy
from model_builders import lstm_builder

Project root 'd:\Project\Command\Python\Neural\Wolf_of_Wall_Street' added to sys.path.
INFO: PyTorch will use GPU.


d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\.venv\Lib\site-packages\pandera\_pandas_deprecated.py:149: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


### 2. API登录、加载配置与初始化

建立与数据源的连接，加载配置文件，并初始化核心服务。

In [2]:
bs.login()

CONFIG_FILE_PATH = 'configs/system_config.yaml'
with open(CONFIG_FILE_PATH, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

# 初始化 RiskManager
risk_manager_instance = risk_manager.RiskManager(db_path='order_history.db')

# 加载 IC 历史
try:
    full_ic_history_df = pd.read_csv('ic_history.csv', parse_dates=['date'])
except FileNotFoundError:
    print("ERROR: 'ic_history.csv' not found. Dynamic weighting will use default values.")
    full_ic_history_df = pd.DataFrame()

login success!


FileNotFoundError: [Errno 2] No such file or directory: 'configs/system_config.yaml'

### 3. 核心：执行预测与融合

遍历所有股票，加载其最新的模型，分别进行预测，然后使用动态加权策略进行融合。

In [ ]:
final_signals = []
device = 'cuda' if torch.cuda.is_available() else 'cpu'

for stock_config in tqdm(config['stocks_to_process'], desc='Generating Predictions'):
    ticker = stock_config['ticker']
    # ... (此处省略完整的预测循环代码，但其逻辑与之前版本相同)
    # 它会调用 get_data.get_full_feature_df, 加载模型，然后进行融合
    # 确保所有模块导入路径正确即可

print('\nSUCCESS: All fused predictions have been generated.')

### 4. 执行风险管理与模拟下单

遍历最终的融合信号，通过 `RiskManager` 进行安全审批。

In [ ]:
for signal_info in final_signals:
    # ... (此处省略完整的风控循环代码，与之前版本相同)
    pass

### 5. API 登出与清理

安全地关闭与数据源的连接。

In [ ]:
bs.logout()